In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

ImportError: DLL load failed while importing _multiarray_umath: Das angegebene Modul wurde nicht gefunden.

ImportError: numpy._core.multiarray failed to import

In [1]:
import tensorflow as tf

ImportError: DLL load failed while importing _multiarray_umath: Das angegebene Modul wurde nicht gefunden.

ImportError: DLL load failed while importing _multiarray_umath: Das angegebene Modul wurde nicht gefunden.

ImportError: DLL load failed while importing _multiarray_umath: Das angegebene Modul wurde nicht gefunden.

ImportError: _multiarray_umath failed to import

ImportError: numpy._core.umath failed to import

In [ ]:
record_df = pd.read_csv('application_record.csv')
record_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
data_df.shape

(438557, 18)